# Grid Search on the Law School Dataset

In this example, we look at the well known Law School dataset. We use the preprocessed version of this from `tempeh` which has two feature columns (the normalized LSAT and GPA for each person), a sensitive attribute of race (with values 'white' and 'black') and a label column indicating whether that student passed the bar exam. We start by loading the data:

In [ ]:
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity

from fairlearn.metrics import group_zero_one_loss, group_mean_prediction

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

from tempeh.configurations import datasets
dataset = datasets['lawschool_passbar']()

# Sort out some dimensions
dataset.y_train = dataset.y_train.squeeze()
dataset.y_test = dataset.y_test.squeeze()

We can take a quick look at the data, in particular the number of students who were black and white, and the label column. For the first, we see that the number of white students is more than an order of magnitude larger. Furthermore, almost all students pass the bar exam (although the pass rate for whites is higher):

In [ ]:
l, c = np.unique(dataset.race_train, return_counts=True)
for i in range(len(l)):
    print("Number {0} is {1}".format(l[i], c[i]))

unused = np.ones(len(dataset.y_train))
mean_prediction = group_mean_prediction(unused, dataset.y_train, dataset.race_train)
print("Overall pass rate {0:.3f}".format(mean_prediction.overall))
print("White pass rate   {0:.3f}".format(mean_prediction.by_group['white']))
print("Black pass rate   {0:.3f}".format(mean_prediction.by_group['black']))

## An Unmitigated Predictor

As a point of comparison for later, we can train a predictor without regard to fairness. Recall that the training features in `X` are just LSAT and GPA scores.

In [ ]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True)

unmitigated_predictor.fit(dataset.X_train, dataset.y_train)

With this predictor, we can look at some statistics. First, we examine the normal error and disparity metrics one might expect

In [ ]:
y_pred_unmitigated = unmitigated_predictor.predict(dataset.X_test)

from fairlearn.metrics import group_zero_one_loss, group_mean_prediction

unmitigated_zero_one_loss = group_zero_one_loss(dataset.y_test, y_pred_unmitigated, dataset.race_test)
unmitigated_mean_prediction = group_mean_prediction(dataset.y_test, y_pred_unmitigated, dataset.race_test)

print("Zero One Loss Overall {0:.3f}".format(unmitigated_zero_one_loss.overall))
print("Zero One Loss whites  {0:.3f}".format(unmitigated_zero_one_loss.by_group['white']))
print("Zero One Loss blacks  {0:.3f}".format(unmitigated_zero_one_loss.by_group['black']))
print()
print("Mean Prediction Overall {0:.3f}".format(unmitigated_mean_prediction.overall))
print("Mean Prediction whites  {0:.3f}".format(unmitigated_mean_prediction.by_group['white']))
print("Mean Prediction blacks  {0:.3f}".format(unmitigated_mean_prediction.by_group['black']))

The high general pass rate is obviously causing problems, since this predictor is giving a 100% pass rate for whites and only causing a small error as a result. The disparity (the difference in the mean prediction for blacks and whites) is also very small for similar reasons.

With such unbalanced classes, it is better to use `group_balanced_root_mean_squared_error` to track disparity, since this puts equal weight on over- and under-predictions, regardless of the relative number of either. Similarly, the error is best tracked via the `group_roc_auc_score`:

In [ ]:
from fairlearn.metrics import group_roc_auc_score, group_balanced_root_mean_squared_error

unmitigated_group_roc_auc_score = group_roc_auc_score(dataset.y_test, y_pred_unmitigated, dataset.race_test)
print("Unmitigated Error {0:.3f}".format(1 - unmitigated_group_roc_auc_score.maximum))

unmitigated_group_balanced_rms_error = group_balanced_root_mean_squared_error(dataset.y_test, y_pred_unmitigated, dataset.race_test)
print("Unmitigated Disparity {0:.3f}".format(unmitigated_group_balanced_rms_error.range))

## Doing a Grid Search

We do a grid search in two stages. In the first, we do a low resolution search, with `fairlearn` chosing the grid for us. From this, we identify a region to expand the grid, and do a more detailed sweep around that point.

First the low resolution sweep:

In [ ]:
n_sweep = 9
sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid_size=n_sweep)

sweep.fit(dataset.X_train, dataset.y_train, sensitive_features=dataset.race_train)

print(sweep.best_result.lambda_vec)

Extract the best $\lambda$ value:

In [ ]:
lambda_vec_best = sweep.best_result.lambda_vec
lambda_best = lambda_vec_best[("+", "all", "white")] - lambda_vec_best[("-", "all", "white")]
print("lambda_best =", lambda_best)

Create a higher resolution grid:

In [ ]:
n_second_sweep = 51
second_sweep_multipliers = np.linspace(lambda_best-0.5, lambda_best+0.5, n_second_sweep)

iterables = [['+','-'], ['all'], ['black', 'white']]
midx = pd.MultiIndex.from_product(iterables, names=['sign', 'event', 'group_id'])

second_sweep_lambdas = []
for l in second_sweep_multipliers:
    nxt = pd.Series(np.zeros(4), index=midx)
    if l < 0:
        nxt[("-", "all", "white")] = abs(l)
    else:
        nxt[("+", "all", "white")] = l
    second_sweep_lambdas.append(nxt)
    
multiplier_df = pd.concat(second_sweep_lambdas, axis=1)

Perform the new search

In [ ]:
second_sweep = GridSearch(LogisticRegression(solver='liblinear', fit_intercept=True),
                   constraints=DemographicParity(),
                   grid=multiplier_df)

second_sweep.fit(dataset.X_train, dataset.y_train, sensitive_features=dataset.race_train)

Now we can do some analysis.

### Naive Analysis

We can look at our results using the zero-one loss and mean prediction metrics. These are not very useful, as we shall see:

In [ ]:
sweep_naive_error = np.zeros(n_second_sweep)
sweep_naive_disparity = np.zeros(n_second_sweep)

for i in range(n_second_sweep):
    preds = second_sweep.all_results[i].predictor.predict(dataset.X_test)
    sweep_naive_error[i] = group_zero_one_loss(dataset.y_test, preds, dataset.race_test).maximum
    sweep_naive_disparity[i] = group_mean_prediction(dataset.y_test, preds, dataset.race_test).range
    
plt.scatter(sweep_naive_error, sweep_naive_disparity)
plt.xlabel("Error")
plt.ylabel("Disparity")
plt.show()

### Using Alternative Metrics

As noted above, it is better to use the `group_roc_auc_score` metric for the error, and the `group_balanced_root_mean_squared_error` for disparity. With these metrics, we find:

In [ ]:
sweep_error = np.zeros(n_second_sweep)
sweep_disparity = np.zeros(n_second_sweep)

for i in range(n_second_sweep):
    preds = second_sweep.all_results[i].predictor.predict(dataset.X_test)
    sweep_error[i] = 1 - group_roc_auc_score(dataset.y_test, preds, dataset.race_test).maximum
    sweep_disparity[i] = group_balanced_root_mean_squared_error(dataset.y_test, preds, dataset.race_test).range
    
plt.scatter(sweep_error, sweep_disparity)
plt.xlabel("Error")
plt.ylabel("Disparity")
plt.show()

We can also look at how the predictions are varying with the set of generated Lagrange multipliers. What we see is that we are gradually moving to predict that all students pass the bar.

In [ ]:
mean_predictions = [group_mean_prediction(dataset.y_test, x.predictor.predict(dataset.X_test), dataset.race_test)
                   for x in second_sweep.all_results]

for r in ['black', 'white']:
    plt.scatter(second_sweep_multipliers, [x.by_group[r] for x in mean_predictions], label=r)
plt.xlabel("Multiplier")
plt.ylabel("Opportunity")
plt.legend()
plt.show()